In [315]:
from bs4 import BeautifulSoup
import requests
import json

In [316]:
player_summary_url = "http://www.howstat.com/cricket/Statistics/Players/PlayerOverviewSummary.asp?PlayerID=3600"
player_base_odi_url = "http://www.howstat.com/cricket/Statistics/Players/PlayerOverview_ODI.asp?PlayerID="
player_base_t20_url = "http://www.howstat.com/cricket/Statistics/Players/PlayerOverview_T20.asp?PlayerID="
player_base_test_url = "http://www.howstat.com/cricket/Statistics/Players/PlayerOverview.asp?PlayerID="
player_base_ipl_url = "http://www.howstat.com/cricket/Statistics/IPL/PlayerOverview.asp?PlayerID="

URL = [player_base_odi_url, player_base_t20_url, player_base_test_url]

In [275]:
player_id = '3600'

In [303]:
website_html = requests.get(player_base_t20_url + player_id)
soup = BeautifulSoup(website_html.content, 'html.parser')

In [230]:
match_type_mapping = {
    "BorderedBoxTest": "test",
    "BorderedBoxODI": "odi",
    "BorderedBoxT20": "t20",
    "BorderedBoxIPL": "ipl",
    "BorderedBoxOverall": "overall"
}

In [336]:
# Sample Json of Player Summary
def odi_t20_stat(soup):
    match_index_mapping ={
        "batting": {
            "innings": "Innings:",
            "notOuts": "Not Outs:",
            "agrregate": "Aggregate:",
            "highestScore": "Highest Score:",
            "average": "Average:",
            "50s": "50s:",
            "100s": "100s:",
            "ducks": "Ducks:",
            "4s": "4s:",
            "6s": "6s:",
            "scoringRate": "Scoring Rate",
            "BallsFaced": "Balls Faced:"
            },
        "bowling": {
            "ballDeliveries": "Balls:",
            "runsConceded": "Runs Conceded:",
            "wicktes": "Wickets:",
            "bowlingAverage": "Average:",
            "5Wickets": "5 Wickets in Innings:",
            "bestBowling": "Best:",
            "bowlingEconomyRate": "Economy Rate:",
            "bowlingStrikeRate": "Strike Rate:",
        },
        "captaincy": {
            "Matches/Won/Lost:": "Matches/Won/Lost:"
            }
    }

    stats = {
        "batting": {},
        "bowling": {},
        "captaincy": {}
    }

    for key1 in match_index_mapping.keys():
        for key2 in match_index_mapping[key1].keys():
            try:
                stats[key1][key2] = soup.find('span', string=match_index_mapping[key1][key2]).find_next('td').text.strip()
            except AttributeError:
                stats[key1][key2] = None

    return stats


In [333]:

def test_stat(soup):
    match_index_mapping ={
        "batting": {
            "innings": "Innings:",
            "notOuts": "Not Outs:",
            "agrregate": "Aggregate:",
            "highestScore": "Highest Score:",
            "average": "Average:",
            "50s": "50s:",
            "100s": "100s:",
            "ducks": "Ducks:",
            "4s": "4s:",
            "6s": "6s:",
            "scoringRate": "Scoring Rate",
            "BallsFaced": "Balls Faced:"
            },
        "bowling": {
            "ballDeliveries": "Balls:",
            "runsConceded": "Runs Conceded:",
            "wicktes": "Wickets:",
            "bowlingAverage": "Average:",
            "5Wickets": "5 Wickets in Innings:",
            "bestBowling": "Best - Match:",
            "bowlingEconomyRate": "Economy Rate:",
            "bowlingStrikeRate": "Strike Rate:",
        },
        "captaincy": {
            "Matches/Won/Lost:": "Matches/Won/Lost:"
            }
    }

    stats = {
        "batting": {},
        "bowling": {},
        "captaincy": {}
    }

    for key1 in match_index_mapping.keys():
        for key2 in match_index_mapping[key1].keys():
            try:
                stats[key1][key2] = soup.find('span', string=match_index_mapping[key1][key2]).find_next('td').text.strip()
            except AttributeError:
                stats[key1][key2] = None

    return stats


In [337]:
with open("./howstat_json/howstat_players.json", "r") as outfile:
    player_json_data = json.loads(outfile.read())
    print(len(player_json_data))


6740


In [343]:
ALL_PLAYER_STATS = []

counter = 0
for player in player_json_data:
    PLAYER_STAT = {}
    name = False
    player_id = str(player['id'])
    PLAYER_STAT['playerId'] = player['id']
    PLAYER_STAT['country'] = player['country']
    PLAYER_STAT['totalTestMatches'] = player['test_matches']
    PLAYER_STAT['totalODIMatches'] = player['odi_matches']
    PLAYER_STAT['totalT20Matches'] = player['t20_matches']
    PLAYER_STAT['dob'] = player['dob']

    if player['test_matches'] > 0:
        website_html = requests.get(player_base_test_url + player_id)
        website_soup = BeautifulSoup(website_html.content, 'html.parser')
        PLAYER_STAT['test'] = test_stat(website_soup)
        if not name:
            try:
                PLAYER_STAT['name'] = website_soup.find('td', string="Full Name:").find_next('td').text.strip()
                PLAYER_STAT['battingStyle'] = website_soup.find('td', string="Bats:").find_next('td').text.strip()
                PLAYER_STAT['bowlingStyle'] = website_soup.find('td', string="Bowls:").find_next('td').text.strip()
                name = True
            except:
                print("ERROR ERROR ERROR ERROR ERROR ERROR", player_id)
                continue
    
    if player['odi_matches'] > 0:
        website_html = requests.get(player_base_odi_url + player_id)
        website_soup = BeautifulSoup(website_html.content, 'html.parser')
        PLAYER_STAT['odi'] = odi_t20_stat(website_soup)
        if not name:
            try:
                PLAYER_STAT['name'] = website_soup.find('td', string="Full Name:").find_next('td').text.strip()
                PLAYER_STAT['battingStyle'] = website_soup.find('td', string="Bats:").find_next('td').text.strip()
                PLAYER_STAT['bowlingStyle'] = website_soup.find('td', string="Bowls:").find_next('td').text.strip()
                name = True
            except:
                print("ERROR ERROR ERROR ERROR ERROR ERROR ERROR", player_id)
                continue
    
    if player['t20_matches'] > 0:
        website_html = requests.get(player_base_t20_url + player_id)
        website_soup = BeautifulSoup(website_html.content, 'html.parser')
        PLAYER_STAT['t20'] = odi_t20_stat(website_soup)
        if not name:
            try:
                PLAYER_STAT['name'] = website_soup.find('td', string="Full Name:").find_next('td').text.strip()
                PLAYER_STAT['battingStyle'] = website_soup.find('td', string="Bats:").find_next('td').text.strip()
                PLAYER_STAT['bowlingStyle'] = website_soup.find('td', string="Bowls:").find_next('td').text.strip()
                name = True
            except:
                print("ERROR ERROR ERROR ERROR ERROR ERROR", player_id)
                continue
    
    ALL_PLAYER_STATS.append(PLAYER_STAT)
    print(counter, player_id)
    counter += 1

0 6329
1 2602
2 6924
3 4480
4 6954
5 1653
ERROR ERROR ERROR ERROR ERROR ERROR 869
6 3916
7 5987
8 2603
9 6305


KeyboardInterrupt: 

In [322]:
player_json_data[0]

{'name': 'aaftab alam khan',
 'id': 6329,
 'dob': '31/01/1986',
 'country': 'malta',
 'test_matches': 0,
 'odi_matches': 0,
 't20_matches': 27}